In [1]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

In [2]:
import os
from datetime import date, timedelta
import pandas as pd
from information.terms_info import tags

pd.set_option('display.max_columns', None)
print(f'{tags}')

['ecq', 'face-shield', 'Frontliners', 'masks', 'Quarantine', 'social-distancing', 'work-from-home', 'contact-tracing']


In [3]:
"""
 Declare data scope
"""
# start_date = date.fromisoformat('2020-01-01')
start_date = date.fromisoformat('2020-03-16')
end_date   = date.fromisoformat('2021-02-15')

end_dates = []
current_date = start_date
while (current_date <= end_date):
    end_dates.append(str(current_date).replace('-', ' '))
    current_date += timedelta(days=1)
print(f"{len(end_dates)=}")

len(end_dates)=337


In [4]:
for tag in tags:
    normalize_tag = tag.replace('/', '.')
    #rsv_directory = os.path.abspath()
    msv_df_csv_path = os.path.abspath(f'./../msvs/processed/{normalize_tag}.csv')
    msv_df_pkl_path = os.path.abspath(f'./../msvs/processed/{normalize_tag}.pkl')

    msv_df = None

    for current_date in end_dates:
        current_rsv_path = os.path.join(f'C:/Users/Cheska Hung/Desktop/Thesis-GT Infodemiology/covid-19-google-trends-network/scripts/msvs/data/{normalize_tag}/', f"{current_date}.csv")
        rsv_df = pd.read_csv(current_rsv_path, parse_dates=['date'])
        rsv_df.drop(columns=['isPartial'], inplace=True)
        rsv_df.columns = ['date', 'index']

        if msv_df is None:
            msv_df = rsv_df.copy()
        else:
            # Join data into one
            merged_df = msv_df.merge(rsv_df, on='date', how='outer')

            # Calculate correction factors
            merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
            merged_df['correction_factor'] = merged_df['correction_factor'].fillna(1)

            aggregated_correction_factor = merged_df['correction_factor'].mean()

            # Resclae the new RSV
            merged_df['index_y'] = merged_df['index_y'] * aggregated_correction_factor

            # Fill new data into previous RSV/MSV
            merged_df['index_x'] = merged_df['index_x'].fillna(merged_df['index_y'])
            print(merged_df.tail())
            
            msv_df = merged_df[['date', 'index_x']].copy()
            msv_df.columns = ['date', 'index']

        print(rsv_df.describe())
        
    msv_df.to_csv(msv_df_csv_path, index=False)
    msv_df.to_pickle(msv_df_pkl_path)

                      date       index
count                   31   31.000000
mean   2020-03-31 00:00:00   27.129032
min    2020-03-16 00:00:00    0.000000
25%    2020-03-23 12:00:00    5.500000
50%    2020-03-31 00:00:00   23.000000
75%    2020-04-07 12:00:00   40.000000
max    2020-04-15 00:00:00  100.000000
std                    NaN   26.267524
         date  index_x  index_y  correction_factor
27 2020-04-12     41.0     41.0                1.0
28 2020-04-13     54.0     54.0                1.0
29 2020-04-14     52.0     52.0                1.0
30 2020-04-15     63.0     63.0                1.0
31 2020-04-16     65.0     65.0                1.0
                      date       index
count                   31   31.000000
mean   2020-04-01 00:00:00   29.225806
min    2020-03-17 00:00:00    0.000000
25%    2020-03-24 12:00:00    7.000000
50%    2020-04-01 00:00:00   26.000000
75%    2020-04-08 12:00:00   42.000000
max    2020-04-16 00:00:00  100.000000
std                    NaN   26

C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values


         date  index_x  index_y  correction_factor
33 2020-04-18     57.0     57.0                1.0
34 2020-04-19     74.0     74.0                1.0
35 2020-04-20     87.0     87.0                1.0
36 2020-04-21     92.0     92.0                1.0
37 2020-04-22     91.0     91.0                1.0
                      date       index
count                   31   31.000000
mean   2020-04-07 00:00:00   43.419355
min    2020-03-23 00:00:00    5.000000
25%    2020-03-30 12:00:00   20.000000
50%    2020-04-07 00:00:00   39.000000
75%    2020-04-14 12:00:00   60.000000
max    2020-04-22 00:00:00  100.000000
std                    NaN   29.156788
         date    index_x    index_y  correction_factor
34 2020-04-19  74.000000  24.109338           0.513514
35 2020-04-20  87.000000  29.184989           0.528736
36 2020-04-21  92.000000  30.453901           0.521739
37 2020-04-22  91.000000  30.453901           0.527473
38 2020-04-23  63.445628  63.445628           1.000000
             

C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  m

          date    index_x    index_y  correction_factor
167 2020-08-30   2.010789  11.178503           3.481221
168 2020-08-31   3.032528  22.357006           4.616610
169 2020-09-01   4.063731  23.953935           3.691190
170 2020-09-02   3.505367  11.178503           1.996938
171 2020-09-03  12.775432  12.775432           1.000000
                      date       index
count                   31   31.000000
mean   2020-08-19 00:00:00   22.419355
min    2020-08-04 00:00:00    0.000000
25%    2020-08-11 12:00:00    9.000000
50%    2020-08-19 00:00:00   18.000000
75%    2020-08-26 12:00:00   30.500000
max    2020-09-03 00:00:00  100.000000
std                    NaN   19.916114
          date    index_x    index_y  correction_factor
168 2020-08-31   3.032528  21.924775           4.616610
169 2020-09-01   4.063731  23.490830           3.691190
170 2020-09-02   3.505367  10.962387           1.996938
171 2020-09-03  12.775432  12.528443           0.626202
172 2020-09-04  12.528443  12.528

...

C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  m

          date    index_x    index_y  correction_factor
351 2021-03-02  70.962670  70.907832           1.000526
352 2021-03-03  67.964248  67.911726           1.000526
353 2021-03-04  78.937346  78.897447           1.000794
354 2021-03-05  59.937436  59.922111           1.001044
355 2021-03-06  48.936391  48.936391           1.000000
                      date       index
count                   31   31.000000
mean   2021-02-19 00:00:00   62.258065
min    2021-02-04 00:00:00    0.000000
25%    2021-02-11 12:00:00   50.500000
50%    2021-02-19 00:00:00   67.000000
75%    2021-02-26 12:00:00   76.000000
max    2021-03-06 00:00:00  100.000000
std                    NaN   22.084637
          date    index_x    index_y  correction_factor
352 2021-03-03  67.964248  67.893049           1.000526
353 2021-03-04  78.937346  78.875748           1.000794
354 2021-03-05  59.937436  59.905631           1.001044
355 2021-03-06  48.936391  48.922932           1.001300
356 2021-03-07  84.866311  84.866

C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
C:\Users\Cheska Hung\AppData\Local\Temp\ipykernel_29072\605386066.py:22: RuntimeWarning: invalid value encountered in divide
  m

          date    index_x    index_y  correction_factor
358 2021-03-09  57.882227  57.823581           1.002035
359 2021-03-10  85.803008  85.738413           1.002296
360 2021-03-11  72.814651  72.777956           1.002545
361 2021-03-12  60.829615  60.814456           1.002801
362 2021-03-13   0.000000   0.000000           1.000000
                      date       index
count                   31   31.000000
mean   2021-02-26 00:00:00   62.096774
min    2021-02-11 00:00:00    0.000000
25%    2021-02-18 12:00:00   54.500000
50%    2021-02-26 00:00:00   67.000000
75%    2021-03-05 12:00:00   77.500000
max    2021-03-13 00:00:00  100.000000
std                    NaN   24.806390
          date    index_x    index_y  correction_factor
359 2021-03-10  85.803008  85.717629           1.002296
360 2021-03-11  72.814651  72.760313           1.002545
361 2021-03-12  60.829615  60.799714           1.002801
362 2021-03-13   0.000000   0.000000           1.000000
363 2021-03-14  85.717629  85.717

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Cheska Hung/Desktop/Thesis-GT Infodemiology/covid-19-google-trends-network/scripts/msvs/data/contact-tracing/2020 03 16.csv'